In [1]:
from langchain_community.chat_models import ChatOllama 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage  # Removed invalid import
import retriever
import pandas as pd
import ollama
# Add this import at the top with the others
from langchain_openai import ChatOpenAI
from langchain_community.llms import Ollama

/Users/shivamsinghrawat/Desktop/Assignments/Text Mining/Project/RAG-LLM-Analysis/.venv/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [44]:
def get_template(user_query, context):
    prompt = """
            Using the context provided below, answer the user's question while ensuring the response is related to the U.S. budget. While writing the answer, only provide details relevant to the user question. If the question or context falls outside this scope, politely inform the user that no relevant information was found and suggest rephrasing or changing the question. 
            Context: {context}
            User question: {user_query}
            """
    print(prompt)        
    return prompt
    
def get_llm(model_name):
    if model_name == 'gpt':
        return ChatOpenAI(model="gpt-3.5-turbo", temperature=0) 
    else:
        return ChatOllama(model=model_name, temperature=0)
        
def get_response(user_query, context,model_name):
    llm = get_llm(model_name)
    template = get_template(user_query,context)
    prompt = ChatPromptTemplate.from_template(template)
    print(prompt)
    chain = prompt | llm | StrOutputParser()

    return chain.invoke({
            "context": context,
            "user_query": user_query
            })

In [79]:
query_df2 = pd.read_csv('dataFiles/updated_query_df - Questions1-10.csv')

In [4]:
query_df2.columns

Index(['S.no', 'queries', 'manual_response', 'context', 'gemma_response',
       'deepseek_response', 'llama_response', 'mistral_response',
       'gpt_response'],
      dtype='object')

In [5]:
def process_models(model_name):
    responses=[]
    for i in range(query_df2.shape[0]):
        query = query_df2['queries'][i]
        context = query_df2['context'][i]
        print(f"query: {query}, context: {context}")
        responses.append(get_response(query, context, model_name))
    return responses


In [80]:
query_df2['gpt_response'] = process_models('gpt')

query: How much discretionary funding does the President's 2023 Budget request for USDA?, context: [Document(id='c34913d8-c604-4da7-866a-eeeaaa92c917', metadata={'author': 'Office of Management and Budget', 'creationdate': '2022-03-25T18:00:07-04:00', 'creator': 'Adobe InDesign 17.1 (Windows)', 'moddate': '2022-03-28T10:10:20-04:00', 'page': 48.0, 'page_label': '45', 'producer': 'Adobe PDF Library 16.0.5', 'source': './Documents/BUDGET-2023-BUD.pdf', 'subject': 'Budget of the United States Government, Fiscal Year 2023', 'title': 'Budget of the United States Government, Fiscal Year 2023', 'total_pages': 156.0, 'trapped': '/False'}, page_content='The U.S. Department of Agriculture (USDA) is responsible for providing nutrition assistance \nto low-income Americans and income support for the farm sector, and for conserving and \npreserving the Nation’s forests and private agricultural lands.  The President’s 2023 Budget for \nUSDA:  invests in tackling the climate crisis while mitigating it

In [81]:
query_df2.to_csv('dataFiles/updated_query_df - Questions1-10.csv', index=False)

In [82]:
query_df2['gpt_response']

0    The President's 2023 Budget requests $28.5 bil...
1    The 2023 Budget takes several steps to address...
2    The 2023 Budget strengthens the Nation's suppl...
3    The 2023 Budget allocates more than $18 billio...
4    The 2023 Budget advances U.S. leadership in em...
5    The 2023 Budget proposes $773 billion for the ...
6    The 2023 Budget addresses cybersecurity threat...
7    The 2023 Budget invests $81.7 billion over fiv...
8    The 2023 Budget includes $850 million across H...
9    The 2023 Budget includes $1.1 billion above th...
Name: gpt_response, dtype: object